#  Scipyによる最適化 -第5回-

Scientific computing tools or Python

https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

SciPyは，Pythonによる科学技術計算のためのオープンソースソフトウェアを利用可能な仕組みである。

Scipyは，NumPyパッケージを内部で用いている。具体的には，NumPyで定義された基本的なデータ構造（配列や行列）や，基本的な演算を用いている。また，図示するためにはMatplotlibを用いている。


# 線形計画問題

SciPyの関数`linprog`を用いると，線形計画問題を解くことができる。線形計画問題は，線形不等式・線形等式のもとで，線形関数を最小化または最大化する問題である。線形計画問題は，一般に，次の形で表すことができる。

$$
\begin{array}{ll}
\text{最小化}_x & c^{\top}x \\ 
\text{制約条件} & A_{ub} x \leq b_{ub} \\ 
& A_{eq} x = b_{eq} \\
& \ell \leq x \leq u
\end{array}
$$
ここで，$x$は決定変数ベクトルであり，$c,  b_{ub}, b_{eq}, \ell,u$は定数ベクトルである。また，$A_{ub}, A_{eq}$は定数行列である。

ここでは，次の問題例を扱う。

$$
\begin{array}{ll}
\text{最大化}_{x_1,x_2,x_3,x_4} & 29x_1+45x_2 \\ 
\text{制約条件} & x_1 - x_2 - 3x_3 \leq 5 \\ 
& 2x_1-3x_2-7x_3+3x_4 \geq 10 \\ 
& 2x_1+8x_2+x_3 = 60 \\ 
& 4x_1 + 4x_2 + x_4 = 60 \\
& 0 \leq x_0 \\ 
& 0 \leq x_1 \leq 5 \\ 
& x_2 \leq 0.5 \\ 
& -3 \leq x_3 
\end{array}
$$

この問題例を，`linprog`で解くには，`linprog`の受け取ることのできる形に書き換える必要がある。

まず，問題例では目的が目的関数の最大化だが，`linprog`は目的関数の最小化を実行する。$29x_1+45x_2$を最大化するには，$-29x_1-45x_2$を最小化すればよい。そこで，問題例の目的関数$29x_1+45x_2$の全体に-1をかけて，$-29x_1-45x_2$とする。また，問題例の目的関数には$x_3$と$x_4$の項は含まれないので，これらの係数はともに0とする。これにより，`linprog`の目的関数は，次のものとして表される。

$$
\min_{x_1,x_2,x_3,x_4} -29x_1  -45x_2 + 0 x_3 + 0x_4
$$
決定変数ベクトルを$x=\left[ x_1,x_2,x_3,x_4 \right]^{\top}$と表すと，各変数の係数を表すベクトル$c$は，次のように定められる。
$$
c=\left[ -29, -45,0 ,0 \right]^{\top}
$$

次に制約式を扱う。最初の制約式は，左辺が右辺以下，という不等式制約である。これは，そのままの形で`linprog`の入力とすることができる。2番目の制約式は，左辺が右辺以上，という不等式制約です。これを，左辺が右辺以下の不等式制約として表すために，両辺に-1をかける。これらの2つの不等式制約は，`linprog`の入力可能な形として次のように表される。

$$
\begin{array}{rl}
x_1 - x_2 - 3x_3 + 0x_4 & \leq 5 \\ 
-2x_1+3x_2+7x_3-3x_4 & \leq -10
\end{array}
$$
この2つの制約式を，$A_{ub}x \leq b_{ub}$の形で表すためには，行列とベクトルを次のように定める。
$$
A_{ub} = 
\begin{bmatrix}
1 & -1 & -3 & 0 \\ 
-2 & 3 & 7 & -3
\end{bmatrix},
b_{ub}=
\begin{bmatrix}
5 \\ 
-10
\end{bmatrix}
$$

次に，2つの等式制約を`linprog`の入力形式で表す。
$$
\begin{array}{l}
2x_1+8x_2+1x_3+0x_4 = 60 \\ 
4x_1+4x_2+0x_3+1x_4=60
\end{array}
$$
この2つの等式制約を$A_{eq}x=b_{eq}$の形で表すためには，行列とベクトルを次のように定める。
$$
A_{eq} = 
\begin{bmatrix}
2 & 8 & 1 & 0 \\ 
4 & 4 & 0 & 1
\end{bmatrix},
b_{eq}=
\begin{bmatrix}
60 \\ 
60
\end{bmatrix}
$$

残りの4つの制約式は，4つの変数$x_0,x_1,x_2,x_3$の上下限を定めるものである。`linprog`では，何も指定しなければ変数の下限は0, 上限はない．すなわち，いくらでも大きな値をとることができる非負変数である。この問題例ではこれとは異なる上下限制約を設定する必要がある。

これらの考察を行ったところで，`linprog`によってこの問題例を解くプログラムは次のとおりである。

In [ ]:
import numpy as np
from scipy.optimize import linprog 
c = np.array([-29.0, -45.0, 0.0, 0.0])
A_ub = np.array([[1.0, -1.0, -3.0, 0.0],
                 [-2.0, 3.0, 7.0, -3.0]])
b_ub = np.array([5.0, -10.0])
A_eq = np.array([[2.0, 8.0, 1.0, 0.0],
                [4.0, 4.0, 0.0, 1.0]])
b_eq = np.array([60.0, 60.0])
x0_bounds = (0, None)
x1_bounds = (0, 5.0)
x2_bounds = (-np.inf, 0.5)
x3_bounds = (-3.0, None)
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds]
result = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds=bounds)
print(result)

この結果を見ると，この問題例は実行不能であることがわかる。この問題例で，$x_1$の範囲を$0 \leq x_1 \leq 6$に変更して実行すると，実行可能解が得られる。

In [ ]:
x1_bounds=(0, 6)
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds]
result = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq, bounds = bounds)
print(result)

$x_1$の範囲を変更した問題例は実行可能であり，その最適値は-505.97であることがわかる。最適解は，`result.x`で得られる。

In [ ]:
result.x

この値を，NumPyのデータ構造`array`として表し，`c`との内積を演算子`@`で計算すると，最適値を再現することができる。

In [ ]:
x = np.array(result.x)
print(c @ x)

次に，得られた最適解が，実際に各制約式を満たすことを確認する。例えば，行列`A_ub`とベクトル`x`の内積は，演算子`@`により計算できる。$b_{ub} -A_{ub}x $が非負であれば，制約式が満たされていることがわかる。ここで，`flatten()`は，元の配列を1次元配列に直した配列のコピーを返すものである。

In [ ]:
print(b_ub - (A_ub @ x).flatten())

この結果より，$b_{ub}-A_{ub}x$の値は（浮動小数点計算の範囲で）0であり，不等式制約は満たされていることがわかる。

この計算結果は，既に`result.slack`として計算されており，参照することができる。

In [ ]:
print(result.slack)

次に，等式制約が満たされることを確認する。不等式制約の場合と同様に，$b_{eq}-A_{eq}x$の値を確認する。

In [ ]:
print(b_eq - (A_eq @ x).flatten())

この計算結果より，等式制約は満たされていることがわかる。

この計算結果も，既に`result.con`として計算されている。

In [ ]:
print(result.con)

各変数の上下限制約が満たされていることは，次のプログラムで確認できる。

In [ ]:
print([0 <= result.x[0], 0 <= result.x[1] <= 6.0, result.x[2] <= 0.5, -3.0 <= result.x[3]])

--------

## 課題8


次の線形計画問題Pについて，下記の問いに答えよ。

$$
\begin{array}{ll}
\text{最大化} & 2x_1 - x_2 - x_3  \\ 
\text{制約条件} & 2x_1 + 3x_2 - x_3 \leq 9 \\ 
& 2x_2 + x_3 \geq 4 \\ 
& 3x_1 + 3x_2 + 3x_3 = 6,
\end{array}
$$

(1) 線形計画問題Pを，`linprog`を用いて解け．

(2) 線形計画問題Pの双対問題Dを書け．

(3) 双対問題Dを，`linprog`を用いて解け．

(4) (1)で求めたPの解と(3)で求めたDの解について，相補性定理が成り立つことを確認せよ．

相補性定理については必要に応じて下記を参照せよ．

塩浦昭義，線形計画問題の諸定理，数理計画法第3回講義資料
http://www.dais.is.tohoku.ac.jp/~shioura/teaching/mp12/mp12-03.pdf


--------

In [ ]:
#ここにプログラムを作成してください．

import numpy as np
from scipy.optimize import linprog 
#c = np.array(...)
#A_ub = np.array(...)
#b_ub = np.array(...)
#A_eq = np.array(...)
#b_eq = np.array(...)
x0_bounds = (0, None)
x1_bounds = (0, None)
x2_bounds = (0, None)
#bounds = [...]
#result = linprog(...)
print(result)
print(result.slack)
print(result.con)

# 最適化技術実験 第5回　レポート
### 学生番号 000000 氏名 青学太郎

(ここから下に，レポートを作成してください。レポートを作成したら，「File」->「Save and Checkpoint」でこのjupyter notebookファイルを保存し，さらに，「File」->「Download as」->「Notebook(.ipynb)」を選択して手元のPCにダウンロードしてください．ダウンロードしたファイルの名前を「学生番号-氏名.ipynb」に変更して，CoursePowerから提出してください．ここで，ファイル名内の"学生番号"は1から始まる自分の学生番号に，"氏名"は自分の氏名に置き換えてください．）